# MIT 14.75 Problem Set 5 Question 1

## Setup

In [1]:
library(tidyverse)
library(stargazer)
#  library(AER) # Breaking due to conda issue with survival.

Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.3.0     ✔ purrr   0.3.3
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Please cite as: 

 Hlavac, Marek (2018). stargazer: Well-Formatted Regression and Summary Statistics Tables.
 R package version 5.2.2. https://CRAN.R-project.org/package=stargazer 



In [2]:
load("mss_repdata.RData")

Function for clustered standard errors in `summary()`. See [this post](https://economictheoryblog.com/2018/05/18/cluster-robust-standard-errors-in-stargazer/)

In [3]:
# load necessary packages for importing the function
library(RCurl)
 
# import the function from repository
url_robust <- "https://raw.githubusercontent.com/IsidoreBeautrelet/economictheoryblog/master/robust_summary.R"
eval(parse(text = getURL(url_robust, ssl.verifypeer = FALSE)),
     envir=.GlobalEnv)


Attaching package: ‘RCurl’

The following object is masked from ‘package:tidyr’:

    complete



## (a)

* Column 1: Economic growth rate ~ (growth in rainfall, t) + (growth in rainfall, t-1), clustered by country.
* Column 3: Column 1 + country fixed effects + country-specific time trends
* Column 5: Column 3 + (growth in terms of trade, t)

In [4]:
cclust <- function(m) {
    return(coef(summary(m, cluster = c("country_name")))[, 2])
}

rmse <- function(m, digits=2) {
    return(round(sqrt(mean(m$residuals^2)), digits))
}

In [5]:
fs1 = lm(gdp_g ~ GPCP_g + GPCP_g_l, data=mss_repdata)

In [6]:
fs3 = lm(gdp_g ~ GPCP_g + GPCP_g_l + factor(country_name) +
         factor(country_name) * year, data=mss_repdata)

In [7]:
fs5 = lm(gdp_g ~ GPCP_g + GPCP_g_l + factor(country_name) +
         factor(country_name) * year + tot_100_g, data=mss_repdata)

In [8]:
stargazer(fs1, fs3, fs5,
          se=list(cclust(fs1), cclust(fs3), cclust(fs5)),
          keep=c("GPCP_g", "GPCP_g_l", "tot_100_g"),
          covariate.labels=c("Growth in rainfall, $t$",
                             "Growth in rainfall, $t-1$",
                             "Growth in terms of trade, $t+1$"),
          add.lines=list(c("Country fixed effects",
                           "no", "yes", "yes"),
                         c("Country-specific time trends",
                           "no", "yes", "yes"),
                         c("Root mean square error",
                           rmse(fs1), rmse(fs3), rmse(fs5))),
          keep.stat=c("rsq", "n"),
          dep.var.caption="Dependent Variable: Economic Growth Rate, $t$",
          dep.var.labels="Ordinary Least Squares",
          title="Rainfall and Economic Growth (First-Stage)",
          notes="Standard errors are clustered at the country level.",
          type="text")


Rainfall and Economic Growth (First-Stage)
                                   Dependent Variable: Economic Growth Rate, t     
                              -----------------------------------------------------
                                             Ordinary Least Squares                
                                     (1)                (2)              (3)       
-----------------------------------------------------------------------------------
Growth in rainfall, t              0.055***          0.049***          0.053***    
                                   (0.016)            (0.017)          (0.018)     
                                                                                   
Growth in rainfall, t-1            0.034***           0.028**          0.037**     
                                   (0.013)            (0.014)          (0.015)     
                                                                                   
Growth in terms of trade, t+1   

Table 3

In [9]:
rf1 = lm(any_prio ~ GPCP_g + GPCP_g_l + factor(country_name) +
         factor(country_name) * year, data=mss_repdata)

In [10]:
rf2 = lm(war_prio ~ GPCP_g + GPCP_g_l + factor(country_name) +
         factor(country_name) * year, data=mss_repdata)

In [11]:
stargazer(rf1, rf2,
          se=list(cclust(rf1), cclust(rf2)),
          keep=c("GPCP_g", "GPCP_g_l"),
          covariate.labels=c("Growth in rainfall, $t$",
                             "Growth in rainfall, $t-1$"),
          # Omit is broken, see https://stackoverflow.com/q/38355337/1840471
          omit=c("^(?!.*year).*factor.*$", "year"),
          perl=TRUE,
          omit.labels=c("Country fixed effects",
                        "Country-specific time trends"),
          add.lines=list(c("Root mean square error",
                           rmse(rf1), rmse(rf2))),
          keep.stat=c("rsq", "n"),
#           dep.var.caption="Dependent Variable: Economic Growth Rate, $t$",
          dep.var.labels=c("Civil Conflict >=25 Deaths (OLS)",
                           "Civil Conflict >=1,000 Deaths (OLS)"),
          title="Rainfall and Civil Conflict (Reduced-Form)",
          notes="Standard errors are clustered at the country level.",
          type="text")


Rainfall and Civil Conflict (Reduced-Form)
                                                     Dependent variable:                         
                             --------------------------------------------------------------------
                             Civil Conflict >=25 Deaths (OLS) Civil Conflict >=1,000 Deaths (OLS)
                                           (1)                                (2)                
-------------------------------------------------------------------------------------------------
Growth in rainfall, t                     -0.024                           -0.062**              
                                         (0.043)                            (0.030)              
                                                                                                 
Growth in rainfall, t-1                  -0.122**                          -0.069**              
                                         (0.052)                          

Column 4 and 6 of Table 4

In [12]:
ols = lm(any_prio ~ GPCP_g + GPCP_g_l + factor(country_name) +
         factor(country_name) * year, data=mss_repdata)

In [13]:
stargazer(ols, ols,
          se=list(cclust(ols), cclust(ols)),
          keep=c("GPCP_g", "GPCP_g_l"),
          covariate.labels=c("Growth in rainfall, $t$",
                             "Growth in rainfall, $t-1$"),
          # Omit is broken, see https://stackoverflow.com/q/38355337/1840471
          omit=c("^(?!.*year).*factor.*$", "year"),
          perl=TRUE,
          omit.labels=c("Country fixed effects",
                        "Country-specific time trends"),
          add.lines=list(c("Root mean square error",
                           rmse(rf1), rmse(rf2))),
          keep.stat=c("rsq", "n"),
          dep.var.caption="Dependent Variable: Civil Conflict >=25 Deaths",
          dep.var.labels=c("OLS", "IV-2SLS"),
          title="Economic Growth and Civil Conflict",
          notes="Standard errors are clustered at the country level.",
          type="text")


Economic Growth and Civil Conflict
                                Dependent Variable: Civil Conflict >=25 Deaths   
                             ----------------------------------------------------
                                                     OLS                         
                                        (1)                        (2)           
---------------------------------------------------------------------------------
Growth in rainfall, t                  -0.024                    -0.024          
                                      (0.043)                    (0.043)         
                                                                                 
Growth in rainfall, t-1               -0.122**                  -0.122**         
                                      (0.052)                    (0.052)         
                                                                                 
--------------------------------------------------------------